In [8]:
import tensorflow as tf
import keras
from keras import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Cropping2D
from build_image_generator import image_data_generator
from keras.callbacks import ModelCheckpoint

In [2]:
inputs = Input(shape=(256, 256, 1))

conv1 = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(inputs)
conv2 = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(conv1)
maxp1 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(filters = 128, kernel_size=(3, 3), activation='relu')(maxp1)
conv4 = Conv2D(filters = 128, kernel_size=(3, 3), activation='relu')(conv3)
maxp2 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(filters = 256, kernel_size=(3, 3), activation='relu')(maxp2)
conv6 = Conv2D(filters = 256, kernel_size=(2, 2), activation='relu')(conv5)
maxp3 = MaxPooling2D(pool_size=(2, 2))(conv6)


conv7 = Conv2D(filters = 512, kernel_size=(3, 3), activation='relu')(maxp3)
conv8 = Conv2D(filters = 512, kernel_size=(3, 3), activation='relu')(conv7)

up1 = Conv2DTranspose(filters = 256, kernel_size=(2, 2), strides=(2,2))(conv8)
crop1 = Cropping2D(cropping=(4,4))(conv6)
concat1 = Concatenate(axis=-1)([crop1, up1])
conv9 = Conv2D(filters = 256, kernel_size=(3, 3), activation='relu')(concat1)
conv10 = Conv2D(filters = 256, kernel_size=(3, 3), activation='relu')(conv9)

up2 = Conv2DTranspose(filters = 128, kernel_size=(2, 2), strides=(2,2))(conv10)
crop2 = Cropping2D(cropping=(15,15))(conv4)
concat2 = Concatenate(axis=-1)([crop2, up2])
conv11 = Conv2D(filters = 128, kernel_size=(3, 3), activation='relu')(concat2)
conv12 = Conv2D(filters = 128, kernel_size=(3, 3), activation='relu')(conv11)

up3 = Conv2DTranspose(filters = 64, kernel_size=(2, 2), strides=(2,2))(conv12)
crop3 = Cropping2D(cropping=(38,38))(conv2)
concat3 = Concatenate(axis=-1)([crop3, up3])
conv13 = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(concat3)
conv14 = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(conv13)
output = Conv2D(filters = 1, kernel_size=(1, 1), activation='sigmoid')(conv14)

In [3]:
unet = Model(inputs=inputs, outputs=output)

In [4]:
#print(unet.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 254, 254, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 252, 252, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 126, 126, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [5]:
unet.compile(
    optimizer = 'sgd',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [6]:
batch_size = 4
train_gen = image_data_generator(shuffle=True, batch_size=batch_size)
val_gen = image_data_generator(train=False, batch_size=batch_size)

In [ ]:
checkpointer = ModelCheckpoint('./models/unet_first_model.hdf5', save_best_only=True, verbose=True)

In [7]:
history = unet.fit_generator(
    train_gen,
    steps_per_epoch=10,
    epochs=1,
    verbose=True,
    validation_data=val_gen,
    validation_steps=10
)

Epoch 1/1
10/10 [==============================] - 100s 10s/step - loss: 0.6541 - acc: 0.9348 - val_loss: 0.5987 - val_acc: 0.9802
